# 关于 GDAL 库的补充

栅格数据处理一个很重要的基础库就是 GDAL，有不少现有程序是直接依据该库写的，所以有必要补充了解下其基本内容，官方资料稍微有些晦涩，然而更简易的资料还比较少，能找到的相对较好地如下所示。

参考资料：

- [Python GDAL课程笔记](https://www.osgeo.cn/python_gdal_utah_tutorial/)
- [Python GDAL/OGR Cookbook](https://pcjericks.github.io/py-gdalogr-cookbook/index.html)
- [Open Source Geoprocessing Tutorial](https://github.com/ceholden/open-geo-tutorial)

## GDAL 简介

**GDAL**(Geospatial Data Abstraction Library)是一个开源栅格空间数据转换库。它利用**抽象数据模型**来表达所支持的各种文件格式。还有一系列**命令行工具**来进行数据转换和处理。 **OGR**(OpenGIS Simple Features Reference Implementation)是GDAL项目的一个子项目， 提供对矢量数据的支持。 一般把这两个库合称为**GDAL/OGR**，或者简称为**GDAL**。

很多著名的GIS软件都使用了GDAL/OGR库， 包括商业公司ESRI的ArcGIS，Google的Google Earth和开源的GRASS GIS系统。可以同时对Linux和windows下的地理空间数据管理系统提供百余种矢量和栅格文件类型的支持。

GDAL/OGR使用面向对象的**C++语言**编写，这令该库在支持百余种格式的同时，还具有很高的执行效率。GDAL/OGR同时还提供多种主流编程语言的绑定，比如**Python**。

GDAL提供对多种栅格数据的支持，包括**Arc/Info ASCII Grid(asc)，GeoTiff (tiff)，Erdas Imagine Images(img)，ASCII DEM(dem)** 等格式。

GDAL使用抽象数据模型(abstract data model)来解析它所支持的数据格式，抽象数据模型包括**数据集(dataset)， 坐标系统，仿射地理坐标转换(Affine Geo Transform)， 大地控制点(GCPs)， 元数据(Metadata)，栅格波段(Raster Band)，颜色表(Color Table)， 子数据集域(Subdatasets Domain)，图像结构域(Image_Structure Domain)，XML域(XML:Domains)**。

GDAL包括如下几个部分：

- GDALMajorObject类：带有元数据的对象。
- GDALDdataset类：通常是从一个栅格文件中提取的相关联的栅格波段集合和这些波段的元数据；GDALDdataset也负责所有栅格波段的地理坐标转换(georeferencing transform)和坐标系定义。
- GDALDriver类：文件格式驱动类，GDAL会为每一个所支持的文件格式创建一个该类的实体，来管理该文件格式。
- GDALDriverManager类：文件格式驱动管理类，用来管理GDALDriver类。

OGR提供对矢量数据格式的读写支持，它所支持的文件格式包括：**ESRI Shapefiles， S-57， SDTS， PostGIS，Oracle Spatial， Mapinfo mid/mif ， Mapinfo TAB**。

OGR包括如下几部分：

- Geometry：类Geometry (包括OGRGeometry等类)封装了OpenGIS的矢量数据模型，并提供了一些几何操作，WKB(Well Knows Binary)和WKT(Well Known Text)格式之间的相互转换，以及空间参考系统(投影)。
- Spatial Reference：类OGRSpatialReference封装了投影和基准面的定义。
- Feature：类OGRFeature封装了一个完整feature的定义，一个完整的feature包括一个geometry和geometry的一系列属性。
- Feature Definition：类OGRFeatureDefn里面封装了feature的属性，类型、名称及其默认的空间参考系统等。一个OGRFeatureDefn对象通常与一个层(layer)对应。
- Layer：类OGRLayer是一个抽象基类，表示数据源类OGRDataSource里面的一层要素(feature)。
- Data Source：类OGRDataSource是一个抽象基类，表示含有OGRLayer对象的一个文件或一个数据库。
- Drivers：类OGRSFDriver对应于每一个所支持的矢量文件格式。类OGRSFDriver由类OGRSFDriverRegistrar来注册和管理。

## 用OGR读写矢量数据

In [1]:
try:
    from osgeo import ogr
except:
    import ogr
# 两种导入方式都可以

要读取某种类型的数据，必须先载入数据驱动，即初始化一个对象，让其知道某种数据结构。

In [2]:
driver = ogr.GetDriverByName('ESRI Shapefile')
driver

<osgeo.ogr.Driver; proxy of <Swig Object of type 'OGRDriverShadow *' at 0x000001D6A3E87E40> >

上面可以看到 driver是osgeo.ogr.Driver对象，且是Swig Object的proxy，Simplified Wrapper and Interface Generator ([SWIG](https://en.wikipedia.org/wiki/SWIG#:~:text=The%20Simplified%20Wrapper%20and%20Interface,%2C%20Octave%2C%20Scilab%20and%20Scheme.)) 是一个开源软件工具，用来将C语言或C++写的计算机程序或函式库，连接脚本语言（比如Python）和其它语言，其目的是允许其他编程语言调用用C或C ++编写的函数，允许将复杂的数据类型传递给这些函数，能防止内存被不当释放，能跨语言继承对象类，等等。

driver的open()方法可以返回一个数据源对象，其有两个参数：

```Python
open(<filename>, <update>)
```

filename 文件名，update 为0表示只读，1表示可写。

In [3]:
import sys
filename = 'ospy_data1/sites.shp'
dataSource = driver.Open(filename,0)
if dataSource is None:
    print ('could not open')
    sys.exit()
print ('done!')
dataSource

done!


<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x000001D6A3BD78D0> >

接下来看看矢量图中的数据层。

In [4]:
layer = dataSource.GetLayer(0)
n = layer.GetFeatureCount()
print ('feature count:', n)

feature count: 42


一般获取shapefile的layer时都填0，不填也可。这里 layer 就是shapefile整个全部feature(就是shpfile中的几何形状图)组成的。我个人理解，为什么是0：因为 shpfile 的所有feature都在同一层上，不像栅格图那样有很多不同的bands。

下面代码可以读出 整个shapefile的边界。

In [5]:
extent = layer.GetExtent()
print ('extent:', extent)
print ('ul:', extent[0], extent[3])
print ('lr:', extent[1], extent[2])

extent: (428117.1324650572, 491429.33068615, 4591699.896759847, 4653007.208703939)
ul: 428117.1324650572 4653007.208703939
lr: 491429.33068615 4591699.896759847


如果需要读取其中的某个feature

In [6]:
feat = layer.GetFeature(41)
fid = feat.GetField('id')
print (fid)
feat = layer.GetFeature(0)
fid = feat.GetField('id') #should be a different id
print (fid)

42
1


In [7]:
feat = layer.GetNextFeature()  #读取下一个
# 按顺序读取feature，循环遍历所有的feature
while feat:
    feat = layer.GetNextFeature()
layer.ResetReading()  #复位
feat = layer.GetNextFeature()
feat.GetField('id')

1

下面看feature的几何形状

In [8]:
geom = feat.GetGeometryRef()
geom.GetX()
geom.GetY()
print (geom)

POINT (455552.418360864 4641822.05368488)


接下来看看写数据。

创建新文件主要是使用：

```Python
driver.CreateDataSource(<filename>)
```

filename文件不能是已经存在的，否则会报错。

创建新文件后，要给其创建新的layer

```Python
dataSource.CreateLayer(<name>,geom_type=OGRwkbGeometryType>)
```

OGRwkbGeometryType 中的[wkb](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry)是指 well-known binary，它是Well-known text representation of geometry，一种代表vector geometry的文本标记语言的二进制等价形式，这种二进制形式可以用来以一种更压缩的形式存储和传输和wkt同样的信息以方便计算机处理，当然就不那么人类可读了。

OGRwkbGeometryType 是 OGR 头文件中有声明的一个 枚举类型，列出了well-known binary geometry types。看个例子就更清楚了：

In [9]:
import os
new_file = "ospy_data1/test.shp"
if os.path.isfile(new_file):
    driver.DeleteDataSource(new_file) # TODO: it cannot work
ds2 = driver.CreateDataSource(new_file)
layer2 = ds2.CreateLayer('test', geom_type=ogr.wkbPoint)
ds2

<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x000001D6A3E87F90> >

要添加一个新字段，只能在layer里面加，而且还不能有数据。添加的字段如果是字符串，还要设定宽度。

然后设定几何形状

In [18]:
#create point geometry
pointCoord = -124.4577,48.0135
point = ogr.Geometry(ogr.wkbPoint)
point.AddPoint(pointCoord[0],pointCoord[1])

添加一个新feature，首先得把字段field添加齐。注意OFTString是vector类下的OGRFieldType的一种。

In [19]:
# ogr.FieldDefn(fieldName, fieldType)
fieldName = 'id'
fieldType = ogr.OFTString
fieldDefn = ogr.FieldDefn(fieldName, fieldType)
layer2.CreateField(fieldDefn)

0

然后创建feature，设置其值。

In [20]:
# Create the feature and set values
fieldValue = 'test'
featureDefn = layer2.GetLayerDefn()
outFeature = ogr.Feature(featureDefn)
outFeature.SetGeometry(point)
outFeature.SetField(fieldName, fieldValue)
layer2.CreateFeature(outFeature)

0

这里猜一下 feature，layer，geometry之间的关系。geometry和字段是平级的，他们共同赋予到feature，然后feature在给到layer中。

## 几何形状geometry与投影projection

建立空的geometry对象：ogr.Geometry

定义各种不同的geometry使用的方法是不一样的(point, line, polygon, etc)

新建点point，使用方法AddPoint( <x>, <y>, [<z>])。其中的z坐标一般是省略的，默认值是0

例如：

In [21]:
from osgeo import ogr
point = ogr.Geometry(ogr.wkbPoint)
point.AddPoint(10,20)

新建一个line。使用AddPoint(<x>, <y>, [<z>])添加点；使用SetPoint(<index>, <x>, <y>, [<z>])更改点的坐标

In [22]:
line = ogr.Geometry(ogr.wkbLineString)
line.AddPoint(10,10)
line.AddPoint(20,20)
line.SetPoint(0,30,30)   #(10,10) -> (30,30)
print (line.GetPointCount())

2


读取0号点的x坐标和y坐标

In [23]:
print (line.GetX(0))
print (line.GetY(0))

30.0
30.0


新建多边形，首先要新建环(ring)，然后把环添加到多边形对象中。

如何创建一个ring？先新建一个ring对象，然后向里面逐个添加点。

In [24]:
ring = ogr.Geometry(ogr.wkbLinearRing)
ring.AddPoint(0,0)
ring.AddPoint(100,0)
ring.AddPoint(100,100)
ring.AddPoint(0,100)
# 结束的时候，用CloseRings关闭ring，或者将最后一个点的坐标设定为与第一个点相同。
ring.CloseRings()
# ring.AddPoint(0,0)

下面一个例子，创建一个方框：

In [25]:
outring = ogr.Geometry(ogr.wkbLinearRing)
outring.AddPoint(0,0)
outring.AddPoint(100,0)
outring.AddPoint(100,100)
outring.AddPoint(0,100)
outring.AddPoint(0,0)

inring = ogr.Geometry(ogr.wkbLinearRing)
inring.AddPoint(25,25)
inring.AddPoint(75,25)
inring.AddPoint(75,75)
inring.AddPoint(25,75)
inring.CloseRings()

polygon = ogr.Geometry(ogr.wkbPolygon)
polygon.AddGeometry(outring)
polygon.AddGeometry(inring)
polygon

<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x000001D6A3C2A4B0> >

总之，要先建立一个polygon对象，然后添加ring。数数polygon能有几个ring:

In [26]:
print (polygon.GetGeometryCount())

2


从polygon中读取ring时，index的顺序和创建polygon时添加ring的顺序相同

In [27]:
polygon.GetGeometryRef(0)

<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x000001D6A3BB6900> >

In [28]:
polygon.GetGeometryRef(1)

<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x000001D6A3EE0F60> >

创建复合几何形状multi geometry，例如MultiPoint, MultiLineString, MultiPolygon。用AddGeometry把普通的几何形状加到复合几何形状中，例如：

In [29]:
multipoint = ogr.Geometry(ogr.wkbMultiPoint)
point = ogr.Geometry(ogr.wkbPoint)
point.AddPoint(10,10)
multipoint.AddGeometry(point)
point.AddPoint(20,20)
multipoint.AddGeometry(point)

0

读取MultiGeometry中的Geometry，方法和从Polygon中读取ring是一样的，可以说Polygon是一种内置的MultiGeometry。

不要删除一个已存在的Feature的Geometry，会把python搞崩溃的。

只能删除脚本运行期间创建的Geometry，比方说手工创建出来的，或者调用其他函数自动创建的。就算这个Geometry已经用来创建别的Feature，你还是可以删除它。

例如：Polygon.Destroy()

关于投影Projections，使用SpatialReference对象

多种多样的Projections，GDAL支持WKT, PROJ.4, ESPG, USGS, ESRI.prj

可以从layer和Geometry中读取Projections，例如：

```Python
spatialRef = layer.GetSpatialRef()
spatialRef = geom.GetSpatialReference()
```

投影信息一般存储在.prj文件中，如果没有这个文件，上述函数返回None

建立一个新的Projection：

首先导入osr库，之后使用osr.SpatialReference()创建SpatialReference对象

之后用下列语句向SpatialReference对象导入投影信息

ImportFromWkt(<wkt>)
ImportFromEPSG(<epsg>)
ImportFromProj4(<proj4>)
ImportFromESRI(<proj_lines>)
ImportFromPCI(<proj>, <units>, <parms>)
ImportFromUSGS(<proj_code>, <zone>)
ImportFromXML(<xml>)

导出Projection，使用下面的语句可以导出为字符串

ExportToWkt()
ExportToPrettyWkt()
ExportToProj4()
ExportToPCI()
ExportToUSGS()
ExportToXML()
    
对一个几何形状Geometry进行投影变换，要先初始化两个Projection，然后创建一个CoordinateTransformation对象，用它来做变换。

In [31]:
import osr
sourceSR = osr.SpatialReference()
print (sourceSR) #empty

In [32]:
sourceSR.ImportFromEPSG(32612) #UTM 12N WGS84
print(sourceSR)

PROJCS["WGS 84 / UTM zone 12N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-111],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32612"]]


In [33]:
targetSR = osr.SpatialReference()
targetSR.ImportFromEPSG(4326) #Geo WGS84
#create coordinate transform to go from UTM to geo
coordTrans = osr.CoordinateTransformation(sourceSR, targetSR)
coordTrans

<osgeo.osr.CoordinateTransformation; proxy of <Swig Object of type 'OSRCoordinateTransformationShadow *' at 0x000001D6A4F95720> >

要在适当的时候编辑Geometry，投影变换之后最好就不要再动了吧。

对一个数据源DataSource里面的所有Geometry做投影变换，你得一个一个来。下面是一个例子

In [35]:
driver = ogr.GetDriverByName('ESRI Shapefile')
ds = driver.Open('ospy_data1/sites.shp')
layer = ds.GetLayer()
sr = layer.GetSpatialRef() #UTM 12N WGS84
print (sr)

PROJCS["WGS 84 / UTM zone 12N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-111],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32612"]]


In [38]:
import osr
sr2 = osr.SpatialReference()
sr2.ImportFromEPSG(4326) #unprojected WGS84
ct = osr.CoordinateTransformation(sr, sr2)
feature = layer.GetFeature(0)
geom = feature.GetGeometryRef()
print (geom) #point coords in UTM
geom.Transform(ct)
print (geom) #unprojected point coords

POINT (455552.418360864 4641822.05368488)
POINT (41.9271075694626 -111.536080100302)


将投影写入.prj文件，其实很简单。首先MorphToESRI()，转成字符串，然后开个文本文件往里面写就行了。例如：

In [42]:
sr2.MorphToESRI()
file = open('ospy_data1/test.prj', 'w')
file.write(targetSR.ExportToWkt())
file.close()

## 过滤器，简单的空间分析，函数和模块

Layer对象有一个方法叫SetAttributeFilter(<where_clause>)可以将Layer中符合某一条件的Feature过滤出来。设定了Filter之后就可以用GetNextFeature()方法依次取出符合条件的Feature了。SetAttributeFilter(None)可以清楚一个Filter。

In [43]:
from osgeo import ogr
driver = ogr.GetDriverByName('ESRI Shapefile')
ds = driver.Open('ospy_data1/sites.shp')
layer = ds.GetLayer()
layer.GetFeatureCount()

42

In [44]:
layer.SetAttributeFilter("cover = 'shrubs'")

0

In [45]:
layer.GetFeatureCount()

6

In [46]:
layer.SetAttributeFilter(None)
layer.GetFeatureCount()

42

空间过滤器Spatial filters有两种。一种是SetSpatialFilter(<geom>)，过滤某一类型的Feature，例如参数中填Polygon，就是选出Layer中的所有Polygon。
    
另外还有SetSpatialFilterRect(<minx>, <miny>, <maxx>, <maxy>)，参数输入四个坐标，可以选中方框内的Feature
    
SetSpatialFilter(None)一样是清空空间属性过滤器。

In [56]:
ptDS = driver.Open('ospy3_data/sites.shp', 0)
ptLayer = ptDS.GetLayer()
polyDS = driver.Open('ospy3_data/cache_towns.shp')
polyLayer = polyDS.GetLayer()
polyFeature = polyLayer.GetFeature(18)
polyFeature.GetField('name')
poly = polyFeature.GetGeometryRef()
ptLayer.SetSpatialFilter(poly)
print(ptLayer) #should just be one
ptLayer.SetSpatialFilter(None)
print(ptLayer) #everything is back

<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x000001D6A4F6BB40> >
<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x000001D6A4F6BB40> >


更多内容后面再补充。

## 用GDAL读取栅格数据

GDAL原生支持超过100种栅格数据类型，涵盖所有主流GIS与RS数据格式，包括

- ArcInfo grids, ArcSDE raster, Imagine, Idrisi, ENVI, GRASS, GeoTIFF
- HDF4, HDF5
- USGS DOQ, USGS DEM
- ECW, MrSID
- TIFF, JPEG, JPEG2000, PNG, GIF, BMP

In [10]:
from osgeo import gdal

In [11]:
print("GDAL's version is: " + gdal.__version__)
print(gdal)

GDAL's version is: 3.0.4
<module 'osgeo.gdal' from 'D:\\Code\\hydroGIS\\envs\\lib\\site-packages\\osgeo\\gdal.py'>


In [15]:
fn = 'ospy_data4/aster.img'
ds = gdal.Open(fn, 0)
print(ds)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000027FB55D5990> >


读取栅格数据集的x方向像素数，y方向像素数，和波段数

In [17]:
cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount
print('Number of bands in image: {n}\n'.format(n=bands))
print('Image size is: {r} rows x {c} columns\n'.format(r=rows, c=cols))

Number of bands in image: 3

Image size is: 5033 rows x 5665 columns



GeoTransform是一个list，存储着栅格数据集的地理坐标信息

```Python
adfGeoTransform[0] # top left x 左上角x坐标*
adfGeoTransform[1] # w--e pixel resolution 东西方向上的像素分辨率
adfGeoTransform[2] # rotation, 如果北边朝上，地图的旋转角度就是0
adfGeoTransform[3] # top left y 左上角y坐标
adfGeoTransform[4] # rotation, 如果北边朝上，地图的旋转角度就是0
adfGeoTransform[5] # n-s pixel resolution 南北方向上的像素分辨率
```

In [24]:
geotransform = ds.GetGeoTransform()
originX = geotransform[0]
originY = geotransform[3]
pixelWidth = geotransform[1]
pixelHeight = geotransform[5]
print(originX)
print(originY)
print(pixelWidth)
print(pixelHeight)

419976.5
4662422.5
15.0
-15.0


计算某一坐标对应像素的相对位置(pixel offset)，也就是该坐标与左上角的像素的相对位置，按像素数计算，计算公式如下：
$$xOffset = int((x – originX) / pixelWidth)$$
$$yOffset = int((y – originY) / pixelHeight)$$
读取某一像素点的值，需要分两步:

1. 首先读取一个波段(band)：GetRasterBand (< index >)，其参数为波段的索引号 
2. 然后用ReadAsArray( < xoff >, < yoff >, < xsize >, < ysize >)，读出从(xoff,yoff)开始，大小为(xsize,ysize)的矩阵。如果将矩阵大小设为1X1，就是读取一个像素了。但是这一方法只能将读出的数据放到矩阵中，就算只读取一个像素也是一样。例如：

In [30]:
band = ds.GetRasterBand(1)
xOffset = 1
yOffset = 2
data = band.ReadAsArray(xOffset, yOffset, 2, 3)
data

array([[0, 0],
       [0, 0],
       [0, 0]], dtype=uint8)

如果想一次读取一整张图，那么显然就是将offset都设定为0，size设定为整个图幅的size 即可。

可以看到 2 对应 xsize，在选择出的数据中 对应地是列数。也就是说 如果从数据中进一步取某个像素的值，应该用 data[yoff, xoff]。简而言之，就是这里面row对应y轴，col对应x轴。

In [33]:
data[2, 1]

0

如何更有效率的读取栅格数据？显然一个一个的读取效率非常低，将整个栅格数据集都塞进二维数组也不是个好办法，因为这样占的内存还是很多。更好的方法是**按块(block)来存取数据**，只把要用的那一块放进内存。

平铺(tiled)，即栅格数据按block存储。有的格式，例如GeoTiff没有平铺，一行是一个block。Erdas imagine格式则按64x64像素平铺。 如果一行是一个block，那么按行读取是比较节省资源的。 如果是平铺的数据结构，那么设定ReadAsArray()的参数值，让它一次只读入一个block，就是效率最高的方法了。例如：

In [42]:
rows = 13
cols = 11
xBSize = 5
yBSize = 5
for i in range(0, rows, yBSize):
    if i + yBSize < rows:
        numRows = yBSize
    else:
        numRows = rows - i
        for j in range(0, cols, xBSize):
            if j + xBSize < cols:
                numCols = xBSize
            else:
                numCols = colsnumCols = cols - j
            data = band.ReadAsArray(j, i, numCols, numRows)
type(data)

numpy.ndarray

处理栅格数据时，numpy 是一个很常用的工具，也是现在数据的默认格式。

处理栅格数据时，有些numpy的功能是很常用的，比如mask，即输入一个数组和条件，输出一个数组 这类功能。比如统计大于0的像素个数，可以联合运用mask和sum两个函数

In [44]:
import numpy as np
a = np.array([0, 4, 6, 0, 2])
mask = np.greater(a, 0)
np.sum(mask)

3

## 栅格数据的写入及其他常见处理函数

前面是一些读操作，这里看看写操作。

In [48]:
import gdal, gdalconst

新建数据集使用的函数如下所示：

```Python
Create(<filename>, <xsize>, <ysize>, [<bands>], [<GDALDataType>])
```

In [49]:
driver = gdal.GetDriverByName('HFA')
ds = driver.Create('ospy_data4/sample1.img', 3, 3, 1, gdalconst.GDT_Float32)

在上面这条语句的执行过程中，存储空间已经被分配到硬盘上了。接着需要先引入波段对象

In [50]:
band = ds.GetRasterBand(1)

波段对象支持直接写入矩阵，两个参数分别为x向偏移和y向偏移。首先制造一些数据。

In [52]:
import numpy as np
data2 = np.array([ [0,54,100], [87,230,5], [161,120,24] ])
data3 = np.array([ [0,100,23], [78,29,1], [134,245,0] ])
ndvi = (data3 - data2) / (data3 + data2)
ndvi

D:\Code\hydroGIS\envs\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


array([[        nan,  0.2987013 , -0.62601626],
       [-0.05454545, -0.77606178, -0.66666667],
       [-0.09152542,  0.34246575, -1.        ]])

可以留意有分母为0的情况，虽然也能计算，不过还是提前处理下比较好。

In [53]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [54]:
ndvi = (data3 - data2) / (data3 + data2)
ndvi

array([[        nan,  0.2987013 , -0.62601626],
       [-0.05454545, -0.77606178, -0.66666667],
       [-0.09152542,  0.34246575, -1.        ]])

In [55]:
band.WriteArray(ndvi, 0, 0)

0

也可以设定NoData对应的值

In [56]:
band.SetNoDataValue(-99)

0

In [57]:
ND = band.GetNoDataValue()
ND

-99.0

对于栅格图来说，金字塔的建立还是必要的，

In [58]:
gdal.SetConfigOption('HFA_USE_RRD', 'YES')

In [59]:
ds.BuildOverviews(overviewlist=[2,4, 8,16,32,64,128])

0

栅格数据拼接，重投影，切割，转换矢量数据 等都是常见操作，后面见到再补充。最后补充一些关于 GDAL 命令行操作还有FWTools工具的内容。

## GDAL 命令行工具 与 FWTools 简单了解

这部分内容主要参考了以下资料：

- [GDAL 笔记一：GDAL命令行入门](https://www.jianshu.com/p/e48d0a17628c)
- [GDAL/OGR Quickstart](https://live.osgeo.org/en/quickstart/gdal_quickstart.html)
- [GDAL——命令使用专题——ogrinfo命令](https://www.cnblogs.com/eshinex/p/10301738.html)
- [FWTools](http://wiki.gis.com/wiki/index.php/FWTools)

GDAL 工具：

- 通过gdalinfo去浏览图片信息
- 通过gdal_translate去进行格式转换
- 通过gdalwarp去重投影你的数据
- 通过gdal_warp或者gdal_merge.py去拼接你的数据
- 通过gdaltindex去建立shapefile拥有栅格编号

OGR 工具：

- 通过ogrinfo获取关于数据的信息
- 通过ogr2ogr去转换栅格数据的格式

这些工具的使用可以通过在命令行中执行类似 --help命令，--help-general 命令来查看

In [62]:
! ogrinfo --help-general

Generic GDAL utility command options:
  --version: report version of GDAL in use.
  --license: report GDAL license info.
  --formats: report all configured format drivers.
  --format [format]: details of one format.
  --optfile filename: expand an option file into the argument list.
  --config key value: set system configuration option.
  --debug [on/off/value]: set debug level.
  --pause: wait for user input, time to attach debugger
  --locale [locale]: install locale for debugging (i.e. en_US.UTF-8)
  --help-general: report detailed help on general options.


In [63]:
! ogrinfo --help

Usage: ogrinfo [--help-general] [-ro] [-q] [-where restricted_where|@filename]
               [-spat xmin ymin xmax ymax] [-geomfield field] [-fid fid]
               [-sql statement|@filename] [-dialect sql_dialect] [-al] [-rl] [-so] [-fields={YES/NO}]
               [-geom={YES/NO/SUMMARY}] [[-oo NAME=VALUE] ...]
               [-nomd] [-listmdd] [-mdd domain|`all`]*
               [-nocount] [-noextent] [-wkt_format WKT1|WKT2|...]
               datasource_name [layer [layer ...]]


更多命令信息都可以在GDAL官网查询：[GDAL documentation » Programs](https://gdal.org/programs/index.html)。

命令清单可以参考：[dwtkns/gdal-cheat-sheet](https://github.com/dwtkns/gdal-cheat-sheet)

FWTools 是一个开源的GIS 工具箱，由Frank Warmerdam 整合了一些流行的工具：

- OpenEV – A high performance raster/vector desktop data viewer and analysis tool.
- MapServer – A web mapping package.
- GDAL/OGR – A library and set of command line utility applications for reading and writing a variety of geospatial raster (GDAL) and vector (OGR) formats.
- PROJ.4 – A cartographic projections library with command-line utilities.
- OGDI – A multi-format raster and vector reading technology noteworthy for inclusion of support for various military formats including VPF (i.e., VMAP, VITD), RPF (i.e., CADRG, CIB), and ADRG.
- Python programming language

最后的效果就是一个软件，里面有了上面这些工具，方便使用，其中OpenEV 是一个桌面应用程序，其他的基本上是命令行工具。现在应该用这个的不是太多，所以重点了解下 GDAL命令行即可。